# 4 - Improving Model Performance

http://datascience.computingpatterns.com/

## Add better features

In [1]:
%run 4.py

In [2]:
train = load_feature_matrix("../input/train.csv")
mytrain, mytest = train_test_split(train, test_size = .4)

In [3]:
columns = ["ReputationAtPostCreation", "OwnerUndeletedAnswerCountAtPostTime", "BodyMarkdownLength", "TitleLength"]

In [4]:
features_train, features_test, new_columns = expand_with_bow_features(mytrain, mytest, ["Title"], max_features=200)

In [5]:
new_columns[:10]

['Titleare',
 'Titlefor',
 'Titlethat',
 'Titleapplication',
 'Titledate',
 'Titlefile',
 'Titleimage',
 'Titledoes',
 'Titleof',
 'Titlehelp']

In [6]:
features_train.shape

(84163, 219)

In [7]:
def get_score(train, test, model, columns):
    model.fit(X=np.asarray(train[columns]), y = np.asarray(train.OpenStatus).transpose())
    predictions = model.predict_proba(np.asarray(test[columns]))[:,1]
    return log_loss(test.OpenStatus.values, predictions)

In [8]:
get_score(mytrain, mytest, LogisticRegression(C=0.1),  ["BodyMarkdownLength", "TitleLength"])

0.68219756386567887

In [9]:
get_score(features_train, features_test,  LogisticRegression(C=0.1), new_columns + ["BodyMarkdownLength", "TitleLength"])

0.61754865972688977

### Suggestions

In [14]:
# Remove stopwords
from sklearn.feature_extraction import stop_words
stop_words.ENGLISH_STOP_WORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [15]:
# Use bigram
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=1)
analyze = bigram_vectorizer.build_analyzer()

In [16]:
analyze('Bi-grams are cool!') == (['bi', 'grams', 'are', 'cool', 'bi grams', 'grams are', 'are cool'])

True

## Optimize model parameters

In [64]:
param_grid = [{'C': [1, 10, 100, 1000]}]
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report

In [65]:
clf = GridSearchCV(LogisticRegression(), 
                   param_grid, 
                   cv=10,
                   scoring="log_loss")

In [66]:
clf.fit(np.array(features_train[columns]), features_train.OpenStatus.values)

GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid=[{'C': [1, 10, 100, 1000]}], pre_dispatch='2*n_jobs',
       refit=True, score_func=None, scoring='log_loss', verbose=0)

In [67]:
clf.grid_scores_

[mean: -0.61878, std: 0.00494, params: {'C': 1},
 mean: -0.61878, std: 0.00495, params: {'C': 10},
 mean: -0.61878, std: 0.00495, params: {'C': 100},
 mean: -0.61879, std: 0.00496, params: {'C': 1000}]

## Use ensemble methods 

In [24]:
clf.get_params()

{'cv': 2,
 'error_score': 'raise',
 'estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'estimator__bootstrap': True,
 'estimator__class_weight': None,
 'estimator__criterion': 'gini',
 'estimator__max_depth': None,
 'estimator__max_features': 'auto',
 'estimator__max_leaf_nodes': None,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__n_estimators': 10,
 'estimator__n_jobs': 1,
 'estimator__oob_score': False,
 'estimator__random_state': None,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'fit_params': {},
 'iid': True,
 'loss_func': None,
 'n_jobs': 1,
 'param_grid': [{'min_

Random forest: ensemble method bagging

Solution: Bagging (Bootstrap aggregating) is a method for creating multiple models (models ensemble) by making different random samples of the original data set. Each of the models is trained with one of those samples. The name bagging is short for bootstrap aggregating. A bootstrap sample is a random sample with replacement. The final result of the different models is done by voting in classification problems or by averaging in regression problems. Bagging works well for instable classifiers, e.g. decision trees and neural networks. A classifier is unstable if small changes in training data lead to significantly different models.

In [9]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators        = 32,     # Number of decision trees
                            min_samples_leaf    = 10,
                            min_samples_split   = 10,
                            n_jobs              = -1)     # Number of cores I'm using in parallel)
rf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=10, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=32, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [10]:
get_score(features_train, features_train,  make_submitter(columns, rf))


0.56904778044730386

In [11]:
get_score(features_tr, features_test, make_submitter(new_columns + ["BodyMarkdownLength", "TitleLength"], rf))

0.59781208661851404